In [ ]:
using Gridap        
using GridapGmsh
using Gridap.Geometry
using Gridap.TensorValues
using Plots
using LinearAlgebra
using  Gridap.Fields
using  Gridap.CellData
using  Gridap.ReferenceFEs  
using  Gridap.Fields

In [ ]:
const ϵ3 = TensorValue{2,2}(0.0,-1.0,1.0,0.0) 
const I2 = TensorValue(1.0,0.0,0.0,1.0)
const I1 = VectorValue(0.0,1.0)
const p = 1.0

In [ ]:
const lsp = 8.0  
const Gc = 0.09
const η = 1e-10

In [ ]:
function axl(W)
-(1/2)*((W)⋅²(ϵ3))
end

function skew(w)
-(ϵ3)*w
end

In [ ]:
function axl_map(W)
-(1/2)*(ϵ3⊙W)
end

In [ ]:
using LinearAlgebra: norm
function  exp_map(A)
    ModA = norm(axl_map(A))
    if ModA == 0
        one(A)
   else
  one(A) + ((sin(ModA))/(ModA))*A + ((1 - cos(ModA))/(ModA*ModA))*(A⋅A)
    end
end

In [ ]:
F(∇u) = I2 + ∇u'
U_bar(Q,F) = Q'⋅F - I2
K(Q) = axl((Q')⋅(∇(Q))')
dK(∇dθ,Q) = (Q')⋅(∇dθ)
DQ(skew_dθ,Q) = skew_dθ⋅Q 
DU_bar(Q,∇du,dθ,∇u) = Q'⋅∇du' + (skew(dθ)⋅Q)'⋅F(∇u)
deg_fun(d) = d*d/(d*d+m*(1-d)*(1+p*(1-d)))   # Written such that d = 0 (damage)
g_dash_s_fun(ϕ) = ((ϕ-1)*(ϕ*(2*p+1)+1)*m)/((ϕ*ϕ*(m*p+1)+(ϕ*(m-2)+1))*((ϕ*ϕ*(m*p+1)+(ϕ*(m-2)+1)))) # Written such that d = 0 (damage)

# Note: User need to create a .msh file before running the code

In [ ]:
model = GmshDiscreteModel("LShapeDomain.msh")
writevtk(model,"LShapeDomain")

In [ ]:
labels = get_face_labeling(model)

In [ ]:
const ν = 0.18
const  E = 20000
const G = E/(2*(1+ν))

In [ ]:
const λ_ps = (E*ν)/((1+ν)*(1-2*ν)) # plane strain
const μ = G
const λ = λ_ps*(2*μ/(λ_ps+2*μ))
const fₜ = 2.7
ψ_crit = 0.0002 
const F_crit = ψ_crit*lsp/(Gc)
const m = 3/(8*F_crit)

In [ ]:
const κ = λ + μ
const lb = 5.0
const γ = 4*μ*lb*lb

In [ ]:
const N = 0.5
const μc = μ*N^2/(1 - N^2)

In [ ]:
function Str_P(Q,∇u)
    _U_bar = U_bar(Q,F(∇u))
    _U_bar_sym = 0.5*(_U_bar+_U_bar')
    _U_bar_asym = 0.5*(_U_bar-_U_bar')
    _U_bar_dev = _U_bar_sym - (1/2)*tr(_U_bar_sym)*I2   # for plane strain
    _P = 2*μ*(_U_bar_dev) + κ*tr(_U_bar)*I2 + 2*μc*_U_bar_asym
    return _P
end

function Str_P_mod(Q,∇u,s_in)
    _gs = deg_fun(s_in)
(_gs+η)*Str_P(Q,∇u)  
end

function DStr_P(Q,∇du,dθ,∇u)
    _DU_bar = DU_bar(Q,∇du,dθ,∇u)
    _DU_bar_sym = 0.5*(_DU_bar+_DU_bar')
    _DU_bar_asym = 0.5*(_DU_bar-_DU_bar')
    _DP = 2*μ*(_DU_bar_sym - (1/2)*tr(_DU_bar_sym)*I2) + κ*tr(_DU_bar)*I2 + 2*μc*_DU_bar_asym 
end

function DStr_P_mod(Q,∇du,dθ,∇u,s_in)
    _gs = deg_fun(s_in)
(_gs+η)*DStr_P(Q,∇du,dθ,∇u)
end

In [ ]:
function Str_P_c(Q_in)
  _K = K(Q_in)
   γ*_K 
end

In [ ]:
function QDP_c_θ(∇dθ)
 QDP_c_θ_out =  γ⋅∇dθ 
    return QDP_c_θ_out
end

In [ ]:
function project(q,model,dΩ,order)
  reffe = ReferenceFE(lagrangian,Float64,order)
  V = FESpace(model,reffe,conformity=:L2)
  a(u,v) = ∫( u*v )*dΩ
  l(v) = ∫( v*q )*dΩ
  op = AffineFEOperator(a,l,V,V)
  qh = solve(op)
  qh
end

In [ ]:
function project_vector(q,model,dΩ,order)
  reffe = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
  V = FESpace(model,reffe,conformity=:L2)
  a(u,v) = ∫( u⊙v )*dΩ
  l(v) = ∫( v⊙q)*dΩ
  op = AffineFEOperator(a,l,V,V)
  qh = solve(op)
  qh
end

In [ ]:
function project_tensor(q,model,dΩ,order)
  reffe = ReferenceFE(lagrangian,TensorValue{2,2,Float64,4},order)
  V = FESpace(model,reffe,conformity=:L2)
  a(u,v) = ∫( u⊙v )*dΩ
  l(v) = ∫( v⊙q)*dΩ
  op = AffineFEOperator(a,l,V,V)
  qh = solve(op)
  qh
end

In [ ]:
order = 1
degree = 2*order

In [ ]:
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

In [ ]:
LoadTagId = get_tag_from_name(labels,"LoadEdge")
Γ_Load = BoundaryTriangulation(model,tags = LoadTagId)
dΓ_Load = Measure(Γ_Load,degree)
n_Γ_Load = get_normal_vector(Γ_Load)

In [ ]:
reffe_rot = ReferenceFE(lagrangian,Float64,order)
V0_rot = TestFESpace(model,reffe_rot;
  conformity=:H1,dirichlet_tags=["BottomEdge"],dirichlet_masks=[(true)])
θh  = zero(V0_rot)

In [ ]:
reffe_PF = ReferenceFE(lagrangian,Float64,order)
V0_PF = TestFESpace(model,reffe_PF;
  conformity=:H1)
U_PF = TrialFESpace(V0_PF)
sh = zero(V0_PF)

In [ ]:
reffe_Disp = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
V0_Disp = TestFESpace(model,reffe_Disp;
          conformity=:H1,
          dirichlet_tags=["BottomEdge","LoadEdge"],
          dirichlet_masks=[(true,true),(false,true)])
uh = zero(V0_Disp)
Y = MultiFieldFESpace([V0_Disp,V0_rot])

In [ ]:
function  new_EnergyState(ψPlusPrev_in,ψhPos_in)
    ψPlus_in = F_crit + F_crit*0.5*(ψhPos_in/F_crit-1 + abs(ψhPos_in/F_crit-1)) 
    if ψPlus_in  >= ψPlusPrev_in
            ψPlus_out = ψPlus_in 
        else
            ψPlus_out = ψPlusPrev_in
    end
    true,ψPlus_out
end

In [ ]:
using Gridap.MultiField

In [ ]:
function Q(θ_in,Q_init)        
Q_upd = exp_map∘(skew(θ_in))⋅Q_init
Q_new = project_tensor(Q_upd,model,dΩ,order)
 return Q_new   
end

In [ ]:
 function ψPos(Q,∇u)
     _K = K(Q)
    _U_bar = U_bar(Q,F(∇u))
    _U_bar_sym = 0.5*(_U_bar+_U_bar')
    _U_bar_asym = 0.5*(_U_bar-_U_bar')
    _U_bar_dev = _U_bar_sym - (1/2)*tr(_U_bar_sym)*I2   # for plane strain
    Pos_Tr_E = 0.5*(tr(_U_bar) + abs(tr(_U_bar)))
    ψPlus = (lsp/Gc)*(μ*(_U_bar_dev⊙_U_bar_dev) + 0.5*κ*Pos_Tr_E*Pos_Tr_E + μc*(_U_bar_asym⊙_U_bar_asym) + 0.5*γ*(_K⊙_K))
    return ψPlus
end

In [ ]:
 function   stepDisp(uh_in,θh_in,uApp,cache,Q_old,sh_in)
nls = NLSolver(
  show_trace=true, method=:newton, iterations = 10, linesearch=BackTracking())
solver = FESolver(nls)
uApp1(x) = VectorValue(0.0,0.0)
uApp2(x) = VectorValue(0.0,uApp)
U_Disp = TrialFESpace(V0_Disp,[uApp1,uApp2])
θ_App1(x) = 0.0
U_Rot = TrialFESpace(V0_rot,θ_App1)
res((u,θ),(v,ϕ)) = ∫( ((∇(v))') ⊙ (Q(θ-θh_in_FE,Q_old)⋅(Str_P_mod∘(Q(θ-θh_in_FE,Q_old),∇(u),sh_in) )))*dΩ - ∫(ϕ⊙((ϵ3)⊙(F∘(∇(u))⋅((Q(θ-θh_in_FE,Q_old)⋅((Str_P_mod∘(Q(θ-θh_in_FE,Q_old),∇(u),sh_in))) )'))))dΩ +  ∫((∇(ϕ))⊙(Q(θ-θh_in_FE,Q_old)⋅((deg_fun(sh_in)+η)*Str_P_c(Q(θ-θh_in_FE,Q_old)))))dΩ

jac_disp((u,θ),(du,dθ),(v,ϕ)) = ∫( ((∇(v))') ⊙ (Q(θ-θh_in_FE,Q_old)⋅(DStr_P_mod(Q(θ-θh_in_FE,Q_old),∇(du),dθ,∇(u),sh_in) )))*dΩ + ∫( ((∇(v))') ⊙ (skew(dθ)⋅Q(θ-θh_in_FE,Q_old)⋅(Str_P_mod∘(Q(θ-θh_in_FE,Q_old),∇(u),sh_in) )))*dΩ 
jac_rot((u,θ),(du,dθ),(v,ϕ)) = ∫((∇(ϕ))⊙((skew(dθ)⋅Q(θ-θh_in_FE,Q_old)⋅((deg_fun(sh_in)+η)*Str_P_c(Q(θ-θh_in_FE,Q_old)))) + ((deg_fun(sh_in)+η)*(QDP_c_θ∘(∇(dθ))))))*dΩ -∫(ϕ⊙((ϵ3)⊙(F∘(∇(u))⋅((skew(dθ)⋅Q(θ-θh_in_FE,Q_old)⋅(Str_P_mod∘(Q(θ-θh_in_FE,Q_old),∇(u),sh_in)) + Q(θ-θh_in_FE,Q_old)⋅(DStr_P_mod(Q(θ-θh_in_FE,Q_old),∇(du),dθ,∇(u),sh_in)))'+ ∇(du)'⋅(Q(θ-θh_in_FE,Q_old)⋅(Str_P_mod∘(Q(θ-θh_in_FE,Q_old),∇(u),sh_in)))' ) )))dΩ 
jac((u,θ),(du,dθ),(v,ϕ)) = jac_disp((u,θ),(du,dθ),(v,ϕ)) + jac_rot((u,θ),(du,dθ),(v,ϕ))
X = MultiFieldFESpace([U_Disp, U_Rot])
Num_Free_DOF = num_free_dofs(X)
free_values = zeros(Num_Free_DOF)
Num_Free_DOF_Disp = num_free_dofs(U_Disp) 
free_values[1:Int(Num_Free_DOF_Disp)] = uh_in
free_values[Int(Num_Free_DOF_Disp)+1:Num_Free_DOF] = θh_in
xh_in = FEFunction(X, free_values)  
θh_in_FE = FEFunction(U_Rot, θh_in)      


op = FEOperator(res,jac,X,Y)
xh, cache = solve!(xh_in,solver,op,cache)
uh_out,θh_out = xh
 Δθh = θh_out-θh_in_FE
Q_old = Q(Δθh,Q_old)
        return get_free_dof_values(uh_out),get_free_dof_values(θh_out), cache, uh_out, θh_out, Q_old
end  

In [ ]:
function  stepPhaseField(ψPlusPrev_in,s_in,cache)
res_PF(s,ϕ) = ∫( (3/4)*lsp*lsp*∇(ϕ)⋅∇(s) - (g_dash_s_fun(1-s))*ψPlusPrev_in*ϕ - (3/8)*ϕ )*dΩ  
op_PF = FEOperator(res_PF,U_PF,V0_PF)
nls = NLSolver(
show_trace=true, method=:newton, iterations = 30, linesearch=BackTracking())
solver = FESolver(nls)

sh_out = FEFunction(U_PF,s_in)
sh_out, cache = solve!(sh_out,solver,op_PF,cache)
  return sh_out, get_free_dof_values(sh_out), cache
end

In [ ]:
using LineSearches: BackTracking

In [ ]:
uApp = 0.0
delu = 3e-3
uAppMax = 0.6
const innerMax = 10
count = 0
sh_in = ones(Float64,num_free_dofs(V0_PF))
sPrev = CellState(1.0,dΩ)
sh = project(sPrev,model,dΩ,order)
cache_pf = nothing
cache = nothing
uh_in = get_free_dof_values(uh) 
θh_in =  get_free_dof_values(θh) 
Q_init = Q(θh,I2)
ψPlusPrev = CellState(F_crit,dΩ)
Load = Float64[]
Displacement = Float64[]
uh_in_FE = uh
while uApp .< uAppMax 

    uApp = uApp .+ delu  
    count = count .+ 1
    print("\n Entering displacemtent step :", float(uApp))
    
       
   for inner = 1:innerMax
        ψhPlusPrev = project(ψPlusPrev,model,dΩ,order)
        e = uh - uh_in_FE
         err = sqrt(sum( ∫( e⊙e )*dΩ ))
        print("\n error = ",float(err))
        uh_in_FE = uh
        sh,sh_in,cache_pf = stepPhaseField(ψhPlusPrev,sh_in,cache_pf)   
    uh_in, θh_in, cache, uh, θh, Q_init = stepDisp(uh_in,θh_in,uApp,cache,Q_init,sh)
         ψhPos_in = ψPos(Q_init,∇(uh))
        update_state!(new_EnergyState,ψPlusPrev,ψhPos_in) 
        if err < 1e-8
            break 
        end  
    end
     Node_Force = sum(∫( n_Γ_Load ⋅ (Str_P_mod(Q_init,∇(uh),sh)) ) *dΓ_Load)
    print("\n Load is :", float(abs(Node_Force[2])))
    push!(Load, abs(Node_Force[2]))
    push!(Displacement, uApp)
    
     if mod(count,5)  == 0
writevtk(Ω,"results_MicroPolarLShaped$count",cellfields=
        ["uh"=>uh ,"sigma"=>Str_P_mod(Q_init,∇(uh),sh),"θh"=>θh,"s"=>sh])
    end
end

In [ ]:
plot(Displacement,Load*0.1)

In [ ]:
using DelimitedFiles
Disp = writedlm("LShapedCohesiveMicroPolarNp5lb5Psi20.csv",  [Displacement Load], ',')